In [3]:
import cv2
import numpy as np
import os
# numbrite tuvastamine
import easyocr
# mängu kontrollimiseks(up, down, left, right)
import pyautogui

# screenshot
from mss import mss
from PIL import Image

# types
from typing import Union, List

# Pildilt mänguala otsimine
Järgmine lahendus töötab kenasti järgmistel veebilehtedel
- https://play2048.co/ 
- https://ristsona.postimees.ee/2048
- https://2048game.com/
- https://www.2048.org/ 

In [1]:
debug = False
debug_folder = "debug_imgs/"
debug_cell_folder = "{debug_folder}/cell_images"
if debug:
    os.makedirs(debug_folder, exist_ok=True)
    os.makedirs(debug_cell_folder, exist_ok=True)

# numbrituvastuse mudel
reader = easyocr.Reader(['en'])


In [4]:
def get_board(src:str="screen.png") -> Union[None, np.ndarray]:

    img = cv2.imread(src)
    # pildi eeltöötlus
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (13,13), 0)
    edges = cv2.Canny(gray, 1, 8)
    kernel = np.ones((1,1), np.uint8)
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
    
    contours, _ = cv2.findContours(closed, cv2.RECURS_FILTER, cv2.CHAIN_APPROX_SIMPLE)
    
    board = None
    for contour in contours:
        # pindala, ei ole ideaalne lahendus, aga töötab
        if cv2.contourArea(contour) > 200000:
            board = contour
            break
    
    # boardi ja debugi pärast on vajalik        
    if board is None:
        return None

    x, y, w, h = cv2.boundingRect(board)
    # loome lauda pildi
    board_image = img[y:y+h, x:x+w]
    
    if debug:
        cv2.imwrite(f"{debug_folder}/processed_img.jpg", closed)
        cv2.imwrite(f"{debug_folder}/board.jpg", board_image)
        cv2.drawContours(img, [board], -1, (0, 255, 0), 3)
        cv2.imwrite(f"{debug_folder}/all_contours.jpg", img)

    return board_image

# Jagab pildi 16 osadeks 4x4

In [5]:
def extract_board_cells(board:np.ndarray) -> List[np.ndarray]:

    # pildi pikkus ja laius
    board_height, board_width, _ = board.shape

    # meie laua dimensioon
    rows = 4
    cols = 4

    # ühe ruudu pikkus ja laius
    cell_height = board_height // rows
    cell_width = board_width // cols

    # if debug:
    #     output_dir = 'cell_images'
    #     os.makedirs(output_dir, exist_ok=True)

    cells_imgs = []
    for row in range(rows):
        for col in range(cols):
            # Kõrgem matemaatika 
            x1 = col * cell_width
            y1 = row * cell_height
            x2 = (col + 1) * cell_width
            y2 = (row + 1) * cell_height

            # salvestame ruudu massiivi
            cell = board[y1:y2, x1:x2]
            cells_imgs.append(cell)
            if debug:
                cell_filename = os.path.join(debug_cell_folder, f'cell_{row}_{col}.jpg')
                cv2.imwrite(cell_filename, cell)
            
    return cells_imgs

# Tuvastab numbri

Selleks kasutame eel treenitud easyocr mudelit, mis saab kenasti selle ülesandega hakkama.

In [6]:
def get_cell_number(img:np.ndarray) -> int:
    # Using EasyOCR to detect text on the image
    # allowlist [0-9], et mudel kontrolliks ainult numbreid
    result = reader.readtext(img,  allowlist ='0123456789')
    if debug:
        print(result)
    return int(result[0][1]) if result else 0

# Loob 4x4 matriksi graafi algoritmi jaoks

In [7]:
def create_4x4_board(board:np.ndarray) -> List[List[int]]:
    board_values = [get_cell_number(img) for img in extract_board_cells(board) ]
    board_state = [
        board_values[0:4], #rida1 
        board_values[4:8], 
        board_values[8:12],
        board_values[12:16] #rida2
    ]
    return board_state

# Mängimiseks vajalikud funktisoonid

In [9]:
# need funktsioonid ei tööta väga hästi colabis ehk tuleb kasutada eraldi python skirpitna
# tuleb veel arusaada kuidas valida width ja height, hetkel need värtused sobivad minu arvuti jaoks
def take_screen_shot() -> np.ndarray:
    mon = {'left': 0, 'top': 0, 'width': 1400 , 'height': 900}
    with mss() as sct:
        screenShot = sct.grab(mon)
        img = Image.frombytes(
            'RGB', 
            (screenShot.width, screenShot.height), 
            screenShot.rgb, 
        )
        open_cv_image = np.array(img)
        # Convert RGB to BGR, kuna cv2 on BGR
        # kuskilt stackoverfloavist võtsin seda
        open_cv_image = open_cv_image[:, :, ::-1].copy()
        cv2.imwrite("screen.png", open_cv_image)
        return open_cv_image

def move(direction:str) -> None:
    directions = {
        "up": lambda:pyautogui.press('up'),
        "down":lambda:pyautogui.press('down'),
        "left":lambda:pyautogui.press('left'),
        "right":lambda:pyautogui.press('right')
    }

    directions[direction]()


# Näide

## Kontrollime kas mängulaud on olemas pildil

In [12]:
board = get_board("test_imgs/webpages/other/20.png")
if board is not None:
    new_board = create_4x4_board(board)
    print(new_board)
else:
    print("board not found")

board not found


## Saame  mängulaua väärtused

In [13]:
board = cv2.imread("test_imgs/epic_board_2.png")
new_board = create_4x4_board(board)
print(new_board)

[[32, 64, 8192, 16384], [16, 128, 4096, 32768], [8, 256, 2048, 65536], [4, 512, 1024, 131072]]


# Testimine


In [14]:
# test dirs
boards_1_path = "test_imgs/boards"
webpages_with_board_path = "test_imgs/webpages/board"
webpages_without_board_path = "test_imgs/webpages/other"

# get_board
webpages_with_board_imgs_paths = os.listdir(webpages_with_board_path) 
webpages_without_board_imgs_paths = os.listdir(webpages_without_board_path) 
boards_1_imgs_paths = os.listdir(boards_1_path) 

for file in webpages_with_board_imgs_paths:
    assert get_board(webpages_with_board_path+"/"+file) is not None

for file in webpages_without_board_imgs_paths:
    assert get_board(webpages_without_board_path+"/"+file) is None
    
# extract_board_cells


board_1_expected = {
    "0.png":[[8, 2,8,4], [4,128,32,8], [2,4,2,4], [2,32,0,0]],
    "2.png":[[8, 2,8,4], [4,128,32,8], [2,4,2,4], [0,2,2,32]],
    "4.png":[[8, 2,8,4], [4,128,32,8], [0,2,4,8], [2,0,4,32]],
    "5.png":[[8, 2,8,4], [4,128,32,8], [2,4,8,0], [2,4,32,2]],
    "6.png":[[8, 2,8,4], [4,128,32,8], [2,2,4,8], [2,4,32,2]],
    "7.png":[[0, 2,8,2], [8,128,32,4], [4,2,4,16], [4,4,32,2]],
    "8.png":[[8, 2,8,2], [8,128,32,4], [2,2,4,16], [0,4,32,2]],
    "9.png":[[8, 2,8,2], [8,128,32,4], [0,4,4,16], [2,4,32,2]],
    "10.png":[[8, 2,8,2], [8,128,32,4], [2,0,8,16], [2,4,32,2]],
    "11.png":[[8, 2,8,2], [8,128,32,4], [2,8,16,2], [2,4,32,2]],
    "12.png":[[0, 2,8,2], [0,128,32,2], [16,8,16,4], [4,4,32,4]],
    "13.png":[[2, 2,8,2], [0,128,32,2], [16,8,16,4], [0,8,32,4]],
    "15.png":[[2, 4,8,2], [128,32,2,2], [16,8,16,4], [8,32,4,0]],
    "16.png":[[2, 4,8,2], [128,32,4,0], [16,8,16,4], [8,32,4,2]],
    "17.png":[[2, 4,8,2], [128,32,4,4], [16,8,16,2], [8,32,4,2]],
}

for file in boards_1_imgs_paths:
   board = cv2.imread(boards_1_path+"/"+file)
   new_board = create_4x4_board(board) 
   assert new_board == board_1_expected[file]

# img with big number
board = cv2.imread("test_imgs/epic_board_2.png")
new_board = create_4x4_board(board) 
assert new_board == [[32,64,8192,16384],[16,128,4096,32768],[8,256,2048,65536],[4,512,1024,131072]]